# Simulated Annealing Demo

This is a demo of simulated annealing algorithm. It is a heuristic algorithm for solving optimization problems. It is inspired by annealing in metallurgy, a technique involving heating and controlled cooling of a material to increase the size of its crystals and reduce their defects. The simulated annealing algorithm is an adaptation of the Metropolis–Hastings algorithm, a Monte Carlo method to generate sample states of a thermodynamic system, invented by the physicist Nicholas Metropolis in 1953. The simulated annealing algorithm was independently proposed by Scott Kirkpatrick, C. Daniel Gelatt Jr., and Mario P. Vecchi in 1983.


In [ ]:
# if using JupyterHub, uncomment the following line
# %matplotlib nbagg

import matplotlib
import numpy as np
from matplotlib.figure import Figure

# if using VSCode, uncomment the following line
# matplotlib.use("TkAgg", force=True)

import matplotlib.animation as animation
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.axes import Axes


def neighbors(S: np.ndarray):
    neighbors = []
    rows, cols = S.shape
    for i in range(rows):
        for j in range(cols):
            if S[i][j]:
                neighbors.append((i, j))
            else:
                c = 0
                for k in [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]:
                    c += (
                        k[0] >= 0
                        and k[0] < S.shape[0]
                        and k[1] >= 0
                        and k[1] < S.shape[1]
                        and S[k[0]][k[1]]
                    )
                if c == 0:
                    neighbors.append((i, j))
    neighbor_states = []
    for neighbor in neighbors:
        new_neighbor = S.copy()
        new_neighbor[neighbor[0]][neighbor[1]] = not new_neighbor[neighbor[0]][
            neighbor[1]
        ]
        neighbor_states.append(new_neighbor)
    return neighbor_states


def update(
    frame: int,
    G: nx.Graph,
    fig: Figure,
    cost_vs_iteration: Axes,
    variations,
    stopped_a,
    costs,
    plots,
    iterations_per_frame: int = 10,
):
    iteration = iterations_per_frame * frame
    print("Iteration:", iteration, end="\r")
    for _ in range(iterations_per_frame):
        if all(stopped[0] for stopped in stopped_a):
            break
        iteration += 1
        for stopped, cost, (name, sa) in zip(stopped_a, costs, (variations)):
            if stopped[0]:
                continue
            stop = sa.update()
            cost.append(sa.cost(sa.S))
            if stop:
                stopped[0] = True
                break

    x = variations[0][1].S.shape[0]
    y = variations[0][1].S.shape[1]
    pos = {i: (i % x, i // y) for i in range(0, x * y)}
    cost_vs_iteration.clear()
    for [stopped], plot, cost, (name, sa) in zip(stopped_a, plots, costs, variations):
        p = cost_vs_iteration.plot(cost, label=name)
        if stopped:
            cost_vs_iteration.axhline(
                y=cost[-1], color=p[0].get_color(), linestyle="--"
            )
        plot.clear()
        plot.set_aspect("equal")
        plot.set_title(
            f"T: {sa.T:.2f} | Cost: {sa.cost(sa.S):.2f} | Best: {sa.cost(sa.S_best):.2f}"
        )
        drawn_nodes = nx.draw_networkx_nodes(
            G,
            pos=pos,
            node_color=np.where(sa.S.reshape(-1), "red", "white"),
            ax=plot,
            node_size=sa.W.reshape(-1) * 10,
        )
        drawn_nodes.set_edgecolor("black")
        nx.draw_networkx_edges(G, pos=pos, ax=plot)
    cost_vs_iteration.set_title("Cost vs Iteration")
    cost_vs_iteration.set_xlabel("Iteration")
    cost_vs_iteration.set_ylabel("Cost")
    cost_vs_iteration.legend()
    fig.tight_layout()

# Implement the following Simulated Annealing algorithm.

Note that if you make a change in the next cell, you must rerun it to update the algorithm.


In [ ]:
from typing import Any

from numpy import dtype


class SimulatedAnnealing:
    def __init__(
        self,
        S: np.ndarray[Any, dtype[np.bool_]],
        W: np.ndarray[Any, dtype[np.float64]],
        k: float = 1.0,
    ):
        # necessary variables
        self.S = S
        self.W = W
        self.S_best = S
        self.T: float = self.initial_temperature()
        self.step: int = 0
        # you may add more parameters if you want
        self.k = k

    def cost(self, S):
        return np.sum(self.W * S)

    # This is the main function that is called to run the algorithm
    def update(self):
        S_prime = self.select_neighbor()
        if self.acceptance_criteria(S_prime):
            self.S = S_prime
            if self.cost(self.S) > self.cost(self.S_best):
                self.S_best = self.S
        else:
            # you may want to do something here
            pass
        self.T = self.update_temperature()
        self.step += 1
        return self.stopping_criteria()

    # decide on the initial temperature
    def initial_temperature(self):
        return 0.0

    # update the temperature
    # returns new temperature as a float
    #
    # consider adding a parameter to allow you to compare the effect of different values
    def update_temperature(self) -> float:
        # implement
        return self.T

    # returns True if the algorithm should stop, False otherwise
    #
    # consider adding some more variables to the class to help you keep track 
    # of how the algorithm is doing
    def stopping_criteria(self) -> bool:
        # implement
        return False

    # should determine whether to accept the new state or not
    def acceptance_criteria(self, S_prime) -> bool:
        # some useful variables
        c_S = self.cost(self.S)
        c_S_prime = self.cost(S_prime)
        # implement
        return True
    

    # select some neighbor of the current state
    # most of the time it is a random neighbor, but there are other ways to do this
    def select_neighbor(self):
        new_neighbors = neighbors(self.S)
        return new_neighbors[np.random.randint(len(new_neighbors))]

# Visulation

To get a live visualization, run the next cell.
You can create list of algorithm variants to run, and they will be ploted versus each other.


In [ ]:
# if the visualization, you may want to increase the number of iterations per frame, and decrease fps
iterations_per_frame = 100
frames_per_second = 30
size = 20

# Create the graph
S = np.full((size, size), False)
W = np.random.rand(size, size) * 10

G = nx.Graph()
G.add_nodes_from(range(0, size**2))
for i in range(0, size):
    for j in range(0, size - 1):
        G.add_edge(j + size * i, (j + 1) + size * i)
        G.add_edge(j * size + i, (j + 1) * size + i)

# Pick the variations of SA to test
variations = [
    ("K=1", SimulatedAnnealing(S, W, k=1)),
    ("K=10", SimulatedAnnealing(S, W, k=10)),
]

# Create the figure
grid_size = (1 + len(variations) // 2, 2)
fig = plt.figure(figsize=(8, 8))
cost_vs_iteration = plt.subplot2grid(grid_size, (0, 0), colspan=2, rowspan=1, fig=fig)
plots = [
    plt.subplot2grid(grid_size, (i // 2 + 1, i % 2), colspan=1, rowspan=1, fig=fig)
    for i in range(len(variations))
]
costs = [[] for _ in variations]
stopped_a = [[False] for _ in variations]


ani = animation.FuncAnimation(
    fig,
    update,
    frames=600,
    interval=int(1000 / frames_per_second),
    repeat=False,
    fargs=(
        G,
        fig,
        cost_vs_iteration,
        variations,
        stopped_a,
        costs,
        plots,
        iterations_per_frame,
    ),
)
if False:  # set to True to save the animation
    writervideo = animation.FFMpegWriter(fps=60)
    ani.save("test.mp4", writer=writervideo)
else:
    plt.show()